In [ ]:
from AlgorithmImports import *

class CrackSpreadAlgorithm(QCAlgorithm):
    def Initialize(self):
        # Strategy Configuration
        self.SetStartDate(2010, 1, 1)  # Start Date
        self.SetEndDate(2023, 12, 31)  # End Date
        self.SetCash(6000)  # Initial Capital 

        # Add futures Symbols Object
        self.cl = self.AddFuture(Futures.Energies.CrudeOilWTI).Symbol
        self.rb = self.AddFuture(Futures.Energies.Gasoline).Symbol
        self.ho = self.AddFuture(Futures.Energies.HeatingOil).Symbol

        # Possible Equities to backtest the strategy
        
        """
        PSX: Phillips 66
        MPC: Marathon Petroleum Corporation
        VLO: Valero Energy
        DINO: HF Sinclair Corp
        AE: Adam Resources & Energy, Inc.
        """   
        
        # Parameter to choose de refiners stock
        refiner_symbol = self.GetParameter("refiner_symbol", "PSX")  # "PSX" como valor predeterminado
        self.refiner_symbol = self.AddEquity(refiner_symbol, Resolution.Daily).Symbol
        
        # Add the refinery symbol objecto to the algorithm
        self.refiner_symbol = self.AddEquity(refiner_symbol, Resolution.Daily).Symbol
        
        # Strategy Parameter
        self.window = int(self.GetParameter("window", 21))  # Ventana de 21 días por defecto
        self.thresh = float(self.GetParameter("thresh", 2))  # Umbral de Z-score
        
        # Initializing Rolling Windows
        self.rb_window = RollingWindow[float](self.window)
        self.ho_window = RollingWindow[float](self.window)
        self.cl_window = RollingWindow[float](self.window)
        self.refiner_window = RollingWindow[float](self.window)
        
        # Variables to calculate zscore and divergences
        self.zscore = 0
        self.last_price = 0

    def OnData(self, data):
        # Make sure data is available
        if not (data.ContainsKey(self.cl) and data.ContainsKey(self.rb) and data.ContainsKey(self.ho) and data.ContainsKey(self.refiner_symbol)):
            return
        
        # Add data to the rolling windows
        self.rb_window.Add(data[self.rb].Close)
        self.ho_window.Add(data[self.ho].Close)
        self.cl_window.Add(data[self.cl].Close)
        self.refiner_window.Add(data[self.refiner_symbol].Close)
        
        # Verify that rolling windows are ready
        if self.rb_window.IsReady and self.ho_window.IsReady and self.cl_window.IsReady and self.refiner_window.IsReady:
            # Calculate crackspread per barrel
            rb_per_barrel = self.rb_window[0] * 42
            ho_per_barrel = self.ho_window[0] * 42
            cl_price = self.cl_window[0]
            crack_spread = (2 * rb_per_barrel + ho_per_barrel - 3 * cl_price) / 3
            
            # Normalize crackspread prices and rank them within the rolling window
            crack_spread_rank = self.PercentileRank(self.rb_window, crack_spread)
            refiner_rank = self.PercentileRank(self.refiner_window, data[self.refiner_symbol].Close)
            
            # Ranks difference and zscore calculation
            rank_spread = refiner_rank - crack_spread_rank
            mean_spread = np.mean([bar for bar in self.refiner_window])
            std_spread = np.std([bar for bar in self.refiner_window])
            self.zscore = (rank_spread - mean_spread) / std_spread
            
            # Generate entry and exit signals
            if self.zscore < -self.thresh and not self.Portfolio[self.refiner_symbol].Invested:
                self.SetHoldings(self.refiner_symbol, 1)  # Take a position
            elif self.zscore > self.thresh and self.Portfolio[self.refiner_symbol].Invested:
                self.Liquidate(self.refiner_symbol)  # Exit a position

    def PercentileRank(self, window, value):
        sorted_window = sorted([bar for bar in window])  # Sort data
        count = sum(1 for x in sorted_window if x <= value)
        percentile_rank = count / len(sorted_window)
        return percentile_rank
